# Ответы на вопросы с помощью RAG

Большие языковые модели (LLM) позволяют разрабатывать развитые вопросо-ответные приложения, которые могут отвечать на вопросы на основе заданных данных.
В основе таких систем лежит методика известная как «генерация с расширенным поиском» (retrieval-augmented generation или RAG).

## Что такое RAG?

RAG - это методика, которая позволяет расширить знания LLM дополнительными данными.

Большие языковые модели имеют представление о различных темах, но их знания ограничены общими данными, доступными на момент обучения модели.
Если вам нужно, чтобы модель была в курсе какой-то специфической информации или информации, которая появилась после ее обучения, вам понадобится предоставить модели соответствующие данные.
Предоставление таких данных и применение их в промпте это и есть RAG — генерация с расширенным поиском.

GigaChain предоставляет компоненты для разработки вопрос-ответных приолжени и поддержки RAG-методики в целом.

:::note

В этом разделе приводится пример разработки вопрос-ответетного приложения, которое использует неструктурированные данные.
Другие примеры использования RAG включают:

* [Ответы на основе SQL-данных](/docs/use_cases/sql/);
* [Ответы на основе кода](/docs/use_cases/code_understanding) (например, Python).

:::

## Архитектура RAG

В общем случае приложение с RAG включает два компонента:

* Конвейер для загрузки данных из источника и их индексирование. Как правило индексирование работает в автономном режиме.
* Цепочка RAG, которая в реальном времени обрабатывает запрос пользователя, извлекает соответствующие данные из индекса и передает их модели.

Процесс генерации ответа на основе необработанных данных можно представить следующим образом:

### Индексирование

1. Загрузка документов с помощью [DocumentLoaders](/docs/modules/data_connection/document_loaders/).
2. Использование [разделителей текста](/docs/modules/data_connection/document_transformers/) для разбивки `Documents` на мелкие фрагменты. Это удобно как для индексации данных, так и для передачи их в модель, поскольку большие фрагменты труднее искать и использовать в рамках контекста модели.
3. Хранение данных с помощью [векторных хранилищ](/docs/modules/data_connection/vectorstores/) и создание [эмбеддингов](/docs/modules/data_connection/text_embedding/) для поиска. 

![index_diagram](../../../static/img/rag_indexing.png)

### Извлечение данных и генерация

1. Извлечение релевантных данных с помощью [ретривера](/docs/modules/data_connection/retrievers/) на основе запроса пользователя.
2. Передача промпта, включающего вопрос пользователя и извлеченные данные, в [чат-модель](/docs/modules/model_io/chat) / [LLM](/docs/modules/model_io/llms/) для генерации ответа.

![retrieval_diagram](../../../static/img/rag_retrieval_generation.png)